In [9]:
import numpy as np
from tensorflow.keras.models import load_model

from tensorflow.keras import layers,models
import librosa
import matplotlib.pyplot as plt

In [3]:
from os import chdir
chdir("../")   # only run this ones else error will occur

In [4]:
X_train_l = np.load('private/npy/X_train_gender.npy')
X_test_l = np.load('private/npy/X_test_gender.npy')
y_train_l = np.load('private/npy/y_train_gender.npy')
y_test_l = np.load('private/npy/y_test_gender.npy')

In [7]:
model_gender = models.Sequential()

model_gender.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 130, 1)))
model_gender.add(layers.MaxPooling2D((2, 2)))

model_gender.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_gender.add(layers.MaxPooling2D((2, 2)))

model_gender.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_gender.add(layers.MaxPooling2D((2, 2)))

model_gender.add(layers.Flatten())

model_gender.add(layers.Dense(512, activation='relu'))  # Increase the number of neurons
model_gender.add(layers.Dense(256, activation='relu'))  # Adding more dense layers
model_gender.add(layers.Dense(128, activation='relu'))  # Adding more dense layers

model_gender.add(layers.Dense(2, activation='softmax'))

model_gender.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


In [8]:
ep=1
history_language = model_gender.fit(X_train_l, y_train_l, epochs=ep, validation_data=(X_test_l, y_test_l), batch_size=32)

301/301 [==============================] - 164s 544ms/step - loss: 1.2608 - accuracy: 0.5960 - val_loss: 0.6659 - val_accuracy: 0.6077


In [11]:
def create_model(depth):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 130, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    for _ in range(depth):
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [12]:
def train_and_plot(depths, epochs):
    # Initialize colors for plotting
    colors = plt.cm.viridis(np.linspace(0, 1, len(depths)))

    plt.figure(figsize=(10, 6))

    for i, depth in enumerate(depths):
        # Create model
        model = create_model(depth)

        # Train model
        history = model_gender.fit(X_train_l, y_train_l, epochs=epochs, validation_data=(X_test_l, y_test_l), batch_size=32)
        # Plot accuracy
        plt.plot(history.history['val_accuracy'], color=colors[i], label=f"Depth {depth}")

    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.title('Validation Accuracy vs. Epoch for Different Model Depths')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


In [13]:
# Define depths for models
depths = [1, 2, 3, 4]  # You can modify this list with different depths

# Define number of epochs
epochs = 5

# Call function to train and plot models
train_and_plot(depths, epochs)

Epoch 1/5
137/301 [============>.................] - ETA: 1:20 - loss: 0.6661 - accuracy: 0.6072

KeyboardInterrupt: 

<Figure size 1000x600 with 0 Axes>

In [56]:
audio_file_path = '/Users/dheemankumar/github/audio-ai/female_eng.wav' # Adjust the path as needed
audio, sample_rate = librosa.load(audio_file_path, sr=22050)  # Load audio with a sample rate of 22050

mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate)

# Convert to decibels (log scale)
mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

mel_spectrogram_db_with_channel = np.expand_dims(mel_spectrogram_db, axis=-1)

input_data = mel_spectrogram_db_with_channel.reshape(1, 128, 130, 1)
# Append the processed audio data and label to the lists
print(input_data.shape)

(1, 128, 130, 1)


In [57]:
ans=model_gender_4.predict(input_data)

1/1 [==============================] - 0s 80ms/step


In [58]:
male=ans[0][0]
female=ans[0][1]

In [59]:
if (male>(1.5*female)):
    print("Gender : Male")
elif (female>(1.5*male)):
    print("Gender : Female")
else:
    print("not conform")
    

Gender : Female
